# Articoli più cercati su wikipedia per ogni mese
Ogni documento è costitutito dal mese e anno di riferimento (dal 2016 al 2020), dalle prime 500 pagine (senza contare pagine propietarie di wikipedia non di nostro interesse) più viste in quel periodo per la wikipedia inglese/internazionale e infine la pagina più vista per ciascun giorno di quel mese.

Ciascuna pagina è rappresentata da un documento json il quale ha come attributi il titolo, il page_id di riferimento e il numero di visualizzazioni nell'arco temporale di riferimento (mese o giorno a seconda dei casi).

I dati sono ottenuti tramite l'apposita [API](https://wikitech.wikimedia.org/wiki/Analytics/AQS/Pageviews) all'end point *pageviews/top/en.wikipedia*.
Il *page_id* è ricavato dall'API di *en.wikipedia*.

## Carico funzioni e librerie necessarie
Alcune funzioni sono presenti nella repository github del progetto al seguente [link](https://github.com/riccardorubini98/wikitrend).

In [ ]:
pip install dnspython

In [2]:
import requests
import time as time
from tqdm.notebook import tqdm
from requests.exceptions import ConnectionError
# Import github repository
!git clone https://github.com/riccardorubini98/wikitrend.git
import sys
sys.path.append("/content/wikitrend/")
import toppage_function as tp

fatal: destination path 'wikitrend' already exists and is not an empty directory.


# Collegamento a MongoDB
Collegamento a MongoDB Atlas e download della collezione contente le pagine più visitate mese per mese.

In [ ]:
# Connect mongodb atlas
from pymongo import MongoClient
access_token = ""
client = MongoClient(access_token)
db = client.get_database('wikitrend')
record = db.page_topic # new collection for single pages

# Creo loop per ottenere le pagine più viste

In [3]:
#### Creo liste per giorni in ciascun mese ####
months = []
for i in range(1,13):
  mese = str(i).zfill(2)
  months.append(mese)
# giorni
day_30 = []
for i in range(1,31):
  giorno = str(i).zfill(2)
  day_30.append(giorno)

day_31 = []
for i in range(1,32):
  giorno = str(i).zfill(2)
  day_31.append(giorno)

day_28=[]
for i in range(1,29):
  giorno = str(i).zfill(2)
  day_28.append(giorno)

day_29=[]
for i in range(1,30):
  giorno = str(i).zfill(2)
  day_29.append(giorno)

In [ ]:
for year in ["2016","2017","2018","2019","2020"]:
    print("Year: ", year)
    for month in tqdm(months):
        top_page = {}
        top_page["_id"] = int(year+month) # es _id september 2020 -> {"_id":202009}
        top_page["year"] = int(year)
        top_page["month"] = int(month)
        # Get first 500 pages
        top_month = tp.get_topart(year,month,"all-days",500)
        # adding ids
        s = requests.session()
        for articolo in tqdm(top_month,desc='Adding ids \n'):
          url = tp.api_id(articolo["article"])
          # make api call
          i=0
          not_found=True
          while i <30 and not_found:
              try:
                  r = s.get(url=url,timeout=20)
                  not_found=False
              except ConnectionError:
                  time.sleep(1)
                  i += 1
          r= r.json()
          # Adding id to page document
          articolo["page_id"] = tp.get_pageid(r)

        top_page["top_month"] = top_month

        # Top page for each day
        top_day = []
        if month in ["01","03","05","07","08","10","12"]:
            top_day = tp.get_topday(year,month,day_31)
            top_page["top_day"] = top_day
        elif month in ["04","06","09","11"]:
            top_day = tp.get_topday(year,month,day_30)
        # anni bisestili
        elif month == "02" and year in ["2016","2020"]:
            top_day = tp.get_topday(year,month,day_29)
        # anni non bisestili
        else:
            top_day = tp.get_topday(year,month,day_28)
        top_page["top_day"] = top_day
        # Upload to MongoDB
        records.insert_one(top_page)